In [12]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from math import sqrt
from sklearn.metrics import mean_squared_error
from statsmodels.iolib.summary2 import summary_col
import itertools
import matplotlib.pyplot as plt

In [2]:
# Read only total_tract_population column from acs_data.csv
tract_pop = pd.read_csv(
    "../data/acs_data.csv",
    usecols=["tract", "total_tract_population"],
    dtype={"total_tract_population": np.float32, "tract": str},
)
tract_pop.columns

Index(['tract', 'total_tract_population'], dtype='object')

# Run Regression 

In [3]:
# Load the data
dtype_dict = {
    "B01003_001E_adj_supply_any_avg_tract": np.float32,
    "B01003_001E_adj_supply_hr_avg_tract": np.float32,
    "B01003_001E_totcost_any_avg_tract": np.float32,
    "state": str,
    "county": str,
    "tract": str,
    "median_household_income": np.float32,
    "gini_index": np.float32,
    "educational_attainment": np.float32,
    "employment_status": np.float32,
    "housing_tenure": np.float32,
    "poverty_status": np.float32,
    "commute_time": np.float32,
    "without_health_insurance": np.float32,
    "white": np.float32,
    "black": np.float32,
    "am_indian_alaska_native": np.float32,
    "asian": np.float32,
    "other_race": np.float32,
    "hispanic_latino": np.float32,
    "density": np.float32,
    "white_share_state": np.float32,
    "black_share_state": np.float32,
    "am_indian_alaska_native_share_state": np.float32,
    "asian_share_state": np.float32,
    "other_race_share_state": np.float32,
    "hispanic_latino_share_state": np.float32,
}


data = pd.read_csv(
    "../data/acs_data_all.csv", dtype=dtype_dict
)  # Replace with the path to your data file

# Merge tract_pop to data
data = data.merge(tract_pop, how="left", on="tract", validate="1:1")

# Scale median_household_income by 10,000
data["median_household_income"] = data["median_household_income"] / 10000

# Scale density by 10,000
data["density"] = data["density"] / 10000

# Rename the variables
data.rename(
    columns={
        "B01003_001E_adj_supply_any_avg_tract": "Average Supply of Any Provider",
        "B01003_001E_adj_supply_hr_avg_tract": "Average Supply of HR Provider",
        "B01003_001E_totcost_any_avg_tract": "Average Total Cost of Any Provider",
        "median_household_income": "Median Household Income",
        "gini_index": "Gini Index",
        "educational_attainment": "Educational Attainment",
        "employment_status": "Employment Status",
        "housing_tenure": "Housing Tenure",
        "poverty_status": "Poverty Status",
        "commute_time": "Commute Time",
        "without_health_insurance": "Without Health Insurance",
        "white": "Share of White Population Tract",
        "black": "Share of Black Population Tract",
        "am_indian_alaska_native": "Share of American Indian/Alaska Native Population Tract",
        "asian": "Share of Asian Population Tract",
        "other_race": "Share of Other Race Population Tract",
        "hispanic_latino": "Share of Hispanic/Latino Population Tract",
        "density": "Population Density",
        "tract_pop": "total_tract_population",
        "white_share_state": "Share of White Population",
        "black_share_state": "Share of Black Population",
        "am_indian_alaska_native_share_state": "Share of American Indian/Alaska Native Population",
        "asian_share_state": "Share of Asian Population",
        "other_race_share_state": "Share of Other Race Population",
        "hispanic_latino_share_state": "Share of Hispanic/Latino Population",
    },
    inplace=True,
)

# calculate state population
state_population = data.groupby("state")["total_tract_population"].transform("sum")

# create tract_weight
data["tract_weight"] = data["total_tract_population"] / state_population

In [4]:
data.head()

,Average Supply of Any Provider,Average Supply of HR Provider,Average Total Cost of Any Provider,state,county,tract,Median Household Income,Gini Index,Educational Attainment,Employment Status,...,Share of Hispanic/Latino Population Tract,Population Density,Share of White Population,Share of Black Population,Share of American Indian/Alaska Native Population,Share of Asian Population,Share of Other Race Population,Share of Hispanic/Latino Population,total_tract_population,tract_weight
0,0.024394,0.007030,140.768204,27,27001,27001770100,4.6548,0.4291,0.142012,0.455980,...,0.000150,0.000218,0.000377,2.696204e-06,0.000008,0.000031,0.000058,0.000008,2240.0,0.000404
1,0.277115,0.000000,49.739838,27,27001,27001770200,4.8580,0.3815,0.084266,0.442195,...,0.000067,0.000226,0.000369,1.258228e-06,0.000017,0.000002,0.000019,0.000004,2284.0,0.000411
2,0.481750,0.081783,147.445908,27,27001,27001770300,4.4543,0.4515,0.155082,0.498758,...,0.000240,0.004462,0.000584,5.032914e-06,0.000007,0.000023,0.000015,0.000013,3483.0,0.000627
3,0.420769,0.028768,91.856232,27,27001,27001770400,4.8708,0.4339,0.121503,0.485145,...,0.000134,0.000295,0.000491,2.696204e-06,0.000025,0.000000,0.000008,0.000007,2970.0,0.000535
4,0.009878,0.000000,59.393177,27,27001,27001790501,5.0110,0.4564,0.087353,0.438879,...,0.000033,0.000295,0.000335,7.189876e-07,0.000000,0.000025,0.000003,0.000002,1905.0,0.000343


In [5]:
def run_regression_analysis(
    covariates_list,
    outcome_variable,
    data,
    model_names,
    weights_variable="total_tract_population",
    scalar=1,
):
    regression_results = {}
    rmse_results = {}

    for covariates, model_name in zip(covariates_list, model_names):
        X = sm.add_constant(data[covariates])
        y = data[outcome_variable] * scalar

        # Check if weights_variable is provided
        if weights_variable:
            # Create weights array
            weights = data[weights_variable]
            model = sm.WLS(y, X, weights=weights)
        else:
            model = sm.OLS(y, X)

        result = model.fit()

        rmse = np.sqrt(mean_squared_error(y, result.predict()))

        # Use model name as a key to store results
        regression_results[model_name] = result
        rmse_results[model_name] = rmse

    return regression_results, rmse_results

### Access to Any Provider

In [6]:
# Define the covariates

# First set of covariates will include racial group shares
first_set = [
    "Share of Black Population",
    "Share of American Indian/Alaska Native Population",
    "Share of Asian Population",
    "Share of Other Race Population",
    "Share of Hispanic/Latino Population",
]

# Second set of covariates will include the first set plus educational attainment, median income
second_set = first_set + [
    "Median Household Income",
    "Educational Attainment",
]

# Third set of covariates will include the second set plus age, population density, and unemployment rate
third_set = second_set + [
    "Gini Index",
    "Employment Status",
    "Housing Tenure",
    "Poverty Status",
    "Commute Time",
    "Without Health Insurance",
]

# The fourth set of covariates will include the third set plus the population density
fourth_set = third_set + ["Population Density"]

# Combine all covariates into a list of lists
covariates_list = [first_set, second_set, third_set, fourth_set]

# Define the outcome variable
outcome_variable = "Average Supply of Any Provider"

# Define the scalar
scalar = 1  # Replace with your desired scalar

# Define model names
model_names = ["First Set", "Second Set", "Third Set", "Fourth Set"]

# Call the function to run the regression analysis
regression_results, rmse_results = run_regression_analysis(
    covariates_list,
    outcome_variable,
    data,
    model_names,
    weights_variable="total_tract_population",
    scalar=scalar,
)

# Generate the summary table
summary_table = summary_col(
    list(regression_results.values()),
    stars=True,
    float_format="%.3f",
    model_names=model_names,
    info_dict={
        "N": lambda x: f"{x.nobs:.0f}",
        "R2": lambda x: f"{x.rsquared:.3f}",
        "Adjusted R2": lambda x: f"{x.rsquared_adj:.3f}",
    },
)

# Convert the summary table to a DataFrame
summary_df1 = pd.DataFrame(summary_table.tables[0])

# Convert RMSE results to a DataFrame
rmse_df = pd.DataFrame(rmse_results, index=["RMSE"])

# Append the RMSE results to the summary DataFrame
summary_df1 = pd.concat([summary_df1, rmse_df])

In [7]:
# Variables are in order
covariate_list_ordered = [
    "Share of Black Population",
    "Share of American Indian/Alaska Native Population",
    "Share of Asian Population",
    "Share of Other Race Population",
    "Share of Hispanic/Latino Population",
    "Median Household Income",
    "Educational Attainment",
    "Gini Index",
    "Employment Status",
    "Housing Tenure",
    "Poverty Status",
    "Commute Time",
    "Without Health Insurance",
    "Population Density",
    "const",
]

# Separate list for non-standard error metrics
metrics_list = ["N", "R2", "Adjusted R2", "RMSE"]

# Last non-empty row name
last_non_empty = ""

# New index
new_index = []

for name in summary_df1.index:
    if name == "":
        new_index.append(last_non_empty + "_se")
    else:
        new_index.append(name)
        last_non_empty = name

# Assign the new index to summary_df1
summary_df1.index = new_index

# Create a new list that alternates between each variable name and its corresponding "_se" version
covariate_list_ordered_extended = [
    item
    for sublist in [[var, var + "_se"] for var in covariate_list_ordered]
    for item in sublist
    if item in summary_df1.index
]

# Add the non-standard error metrics to the list
covariate_list_ordered_extended += metrics_list

# Reindex summary_df1 according to covariate_list_ordered_extended
summary_df_ordered = summary_df1.reindex(covariate_list_ordered_extended)

# Specify the indexes that end with '_se' that you want to replace
indexes_to_replace = [name for name in summary_df_ordered.index if name.endswith("_se")]

# Create a mapping dictionary to replace specified indexes with an empty string
index_replace_dict = {index: "" for index in indexes_to_replace}

# Use the .rename method with the mapping dictionary
summary_df_ordered.rename(index=index_replace_dict, inplace=True)

summary_df_ordered

,First Set,Second Set,Third Set,Fourth Set
Share of Black Population,-66.174,-267.471***,-395.081***,-382.759***
,(91.852),(92.213),(100.668),(102.846)
Share of American Indian/Alaska Native Population,-94.009,-118.957,-52.549,-58.755
,(204.990),(200.741),(217.195),(217.505)
Share of Asian Population,-1.882,-1.119,-1.227,-1.158
,(2.565),(2.498),(2.472),(2.475)
Share of Other Race Population,-0.099,1.123,0.738,0.688
,(2.074),(2.042),(2.023),(2.025)
Share of Hispanic/Latino Population,-102.237,-107.840,-369.365**,-361.579**
,(154.317),(151.159),(163.105),(163.681)


### High Rated Providers

In [8]:
# Define the outcome variable
outcome_variable = "Average Supply of HR Provider"

# Call the function to run the regression analysis
regression_results, rmse_results = run_regression_analysis(
    covariates_list,
    outcome_variable,
    data,
    model_names,
    weights_variable="total_tract_population",
    scalar=scalar,
)

# Generate the summary table
summary_table = summary_col(
    list(regression_results.values()),
    stars=True,
    float_format="%.3f",
    model_names=model_names,
    info_dict={
        "N": lambda x: f"{x.nobs:.0f}",
        "R2": lambda x: f"{x.rsquared:.3f}",
        "Adjusted R2": lambda x: f"{x.rsquared_adj:.3f}",
    },
)

# Convert the summary table to a DataFrame
summary_df1 = pd.DataFrame(summary_table.tables[0])

# Convert RMSE results to a DataFrame
rmse_df = pd.DataFrame(rmse_results, index=["RMSE"])

# Append the RMSE results to the summary DataFrame
summary_df1 = pd.concat([summary_df1, rmse_df])

In [9]:
# Variables are in order of the regression table
covariate_list_ordered = [
    "Share of Black Population",
    "Share of American Indian/Alaska Native Population",
    "Share of Asian Population",
    "Share of Other Race Population",
    "Share of Hispanic/Latino Population",
    "Median Household Income",
    "Educational Attainment",
    "Gini Index",
    "Employment Status",
    "Housing Tenure",
    "Poverty Status",
    "Commute Time",
    "Without Health Insurance",
    "Population Density",
    "const",
]

# Separate list for non-standard error metrics
metrics_list = ["N", "R2", "Adjusted R2", "RMSE"]

# Last non-empty row name
last_non_empty = ""

# New index
new_index = []

for name in summary_df1.index:
    if name == "":
        new_index.append(last_non_empty + "_se")
    else:
        new_index.append(name)
        last_non_empty = name

# Assign the new index to summary_df1
summary_df1.index = new_index

# Create a new list that alternates between each variable name and its corresponding "_se" version
covariate_list_ordered_extended = [
    item
    for sublist in [[var, var + "_se"] for var in covariate_list_ordered]
    for item in sublist
    if item in summary_df1.index
]

# Add the non-standard error metrics to the list
covariate_list_ordered_extended += metrics_list

# Reindex summary_df1 according to covariate_list_ordered_extended
summary_df_ordered = summary_df1.reindex(covariate_list_ordered_extended)

# Specify the indexes that end with '_se' that you want to replace
indexes_to_replace = [name for name in summary_df_ordered.index if name.endswith("_se")]

# Create a mapping dictionary to replace specified indexes with an empty string
index_replace_dict = {index: "" for index in indexes_to_replace}

# Use the .rename method with the mapping dictionary
summary_df_ordered.rename(index=index_replace_dict, inplace=True)

summary_df_ordered

,First Set,Second Set,Third Set,Fourth Set
Share of Black Population,299.071***,236.778***,154.270***,85.528**
,(37.120),(33.991),(36.462),(36.072)
Share of American Indian/Alaska Native Population,-97.675,25.696,-48.284,-13.660
,(82.843),(73.996),(78.669),(76.287)
Share of Asian Population,1.206,0.586,0.483,0.096
,(1.037),(0.921),(0.895),(0.868)
Share of Other Race Population,0.034,-0.916,-1.185,-0.905
,(0.838),(0.753),(0.733),(0.710)
Share of Hispanic/Latino Population,111.706*,213.676***,100.569*,57.132
,(62.365),(55.719),(59.077),(57.409)


### Total Cost

In [10]:
# Define the outcome variable
outcome_variable = "Average Total Cost of Any Provider"

# Call the function to run the regression analysis
regression_results, rmse_results = run_regression_analysis(
    covariates_list,
    outcome_variable,
    data,
    model_names,
    weights_variable="total_tract_population",
    scalar=scalar,
)

# Generate the summary table
summary_table = summary_col(
    list(regression_results.values()),
    stars=True,
    float_format="%.3f",
    model_names=model_names,
    info_dict={
        "N": lambda x: f"{x.nobs:.0f}",
        "R2": lambda x: f"{x.rsquared:.3f}",
        "Adjusted R2": lambda x: f"{x.rsquared_adj:.3f}",
    },
)

# Convert the summary table to a DataFrame
summary_df1 = pd.DataFrame(summary_table.tables[0])

# Convert RMSE results to a DataFrame
rmse_df = pd.DataFrame(rmse_results, index=["RMSE"])

# Append the RMSE results to the summary DataFrame
summary_df1 = pd.concat([summary_df1, rmse_df])

In [11]:
# Variables are in order of the regression table
covariate_list_ordered = [
    "Share of Black Population",
    "Share of American Indian/Alaska Native Population",
    "Share of Asian Population",
    "Share of Other Race Population",
    "Share of Hispanic/Latino Population",
    "Median Household Income",
    "Educational Attainment",
    "Gini Index",
    "Employment Status",
    "Housing Tenure",
    "Poverty Status",
    "Commute Time",
    "Without Health Insurance",
    "Population Density",
    "const",
]

# Separate list for non-standard error metrics
metrics_list = ["N", "R2", "Adjusted R2", "RMSE"]

# Last non-empty row name
last_non_empty = ""

# New index
new_index = []

for name in summary_df1.index:
    if name == "":
        new_index.append(last_non_empty + "_se")
    else:
        new_index.append(name)
        last_non_empty = name

# Assign the new index to summary_df1
summary_df1.index = new_index

# Create a new list that alternates between each variable name and its corresponding "_se" version
covariate_list_ordered_extended = [
    item
    for sublist in [[var, var + "_se"] for var in covariate_list_ordered]
    for item in sublist
    if item in summary_df1.index
]

# Add the non-standard error metrics to the list
covariate_list_ordered_extended += metrics_list

# Reindex summary_df1 according to covariate_list_ordered_extended
summary_df_ordered = summary_df1.reindex(covariate_list_ordered_extended)

# Specify the indexes that end with '_se' that you want to replace
indexes_to_replace = [name for name in summary_df_ordered.index if name.endswith("_se")]

# Create a mapping dictionary to replace specified indexes with an empty string
index_replace_dict = {index: "" for index in indexes_to_replace}

# Use the .rename method with the mapping dictionary
summary_df_ordered.rename(index=index_replace_dict, inplace=True)

summary_df_ordered

,First Set,Second Set,Third Set,Fourth Set
Share of Black Population,158186.019***,188388.462***,177157.530***,146742.032***
,(20895.457),(15621.052),(16703.854),(16561.923)
Share of American Indian/Alaska Native Population,-248551.079***,-91636.640***,-69242.681*,-53922.988
,(46633.459),(34006.085),(36039.256),(35026.081)
Share of Asian Population,1743.729***,629.540,559.416,388.513
,(583.568),(423.168),(410.173),(398.623)
Share of Other Race Population,2297.628***,562.964,455.744,579.625*
,(471.723),(345.877),(335.709),(326.176)
Share of Hispanic/Latino Population,31673.113,153684.045***,112895.248***,93675.771***
,(35105.943),(25606.788),(27064.124),(26358.568)


In [ ]:
# Create a scatter plot of share of black population vs access to any provider
scatter_plot(
    x, y, x_label="Variable 1", y_label="Variable 2", title="Example Scatter Plot"
)